In [1]:
from vllm import LLM, SamplingParams, ModelRegistry
import json
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
!export CUDA_VISIBLE_DEVICES=1
#from vllm.model_executor.models.llama_ee import LlamaForCausalLMEE
#ModelRegistry.register_model("LlamaForCausalLMEE", LlamaForCausalLMEE)
# from huggingface_hub import notebook_login
# notebook_login()

INFO 03-23 02:29:42 __init__.py:183] Automatically detected platform cuda.


In [2]:
def run_batch_inference(llm, input_file, output_file, batch_size=8):
    prompts = []
    with open(input_file, "r") as f:
        for line in f.readlines():
            prompts.append(json.loads(line)["text"])

    sampling_params = SamplingParams(seed=42, max_tokens=100)

    outputs = llm.generate(prompts, sampling_params)

    input_prompt = []
    generated_text = []
    arrival_time = []
    last_token_time = []
    first_scheduled_time = []
    first_token_time = []
    time_in_queue = []
    finished_time = []
    scheduler_time = []
    model_forward_time = []
    model_execute_time = []
    for output in outputs:
        input_prompt.append(output.prompt)
        generated_text.append(output.outputs[0].text)
        metrics = output.metrics
        arrival_time.append(metrics.arrival_time)
        last_token_time.append(metrics.last_token_time)
        first_scheduled_time.append(metrics.first_scheduled_time)
        first_token_time.append(metrics.first_token_time)
        time_in_queue.append(metrics.time_in_queue)
        finished_time.append(metrics.finished_time)
        scheduler_time.append(metrics.scheduler_time)
        model_forward_time.append(metrics.model_forward_time)
        model_execute_time.append(metrics.model_execute_time)
    df = pd.DataFrame({
        "input_prompt": input_prompt,
        "generated_text": generated_text,
        "arrival_time": arrival_time,
        "last_token_time": last_token_time,
        "first_scheduled_time": first_scheduled_time,
        "first_token_time": first_token_time,
        "time_in_queue": time_in_queue,
        "finished_time": finished_time,
        "scheduler_time": scheduler_time,
        "model_forward_time": model_forward_time,
        "model_execute_time": model_execute_time
    })
    df.to_csv(output_file, index=False)

In [3]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=500)


In [4]:
llm = LLM(model="meta-llama/Llama-2-7b-chat-hf", max_model_len=1024, enforce_eager=True)

INFO 03-23 02:29:57 config.py:520] This model supports multiple tasks: {'embed', 'generate', 'reward', 'classify', 'score'}. Defaulting to 'generate'.
WARNING 03-23 02:29:57 cuda.py:100] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 03-23 02:29:57 config.py:656] Async output processing is not supported on the current platform type cuda.
INFO 03-23 02:29:57 llm_engine.py:232] Initializing an LLM engine (v0.7.0) with config: model='meta-llama/Llama-2-7b-chat-hf', speculative_config=None, tokenizer='meta-llama/Llama-2-7b-chat-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 03-23 02:30:07 model_runner.py:1115] Loading model weights took 12.5523 GB
input_ids shape before converting: torch.Size([2048])
input_ids shape after converting: torch.Size([2048])
LlamaModel forward. input_ids shape: torch.Size([2048])
attention_mask shape: None
-------------------------
Is first rank. input ids shape: torch.Size([2048])
[compute_logits] hidden_states shape: torch.Size([2048, 4096]). lm_head shape: ParallelLMHead(num_embeddings=32000, embedding_dim=4096, org_vocab_size=32000, num_embeddings_padded=32000, tp_size=1)
[compute_logits] logits: tensor([[ 0.4258,  0.3872,  1.4443,  ..., -3.8848,  1.9688, -1.6094],
        [ 0.4258,  0.3872,  1.4443,  ..., -3.8848,  1.9688, -1.6094],
        [ 0.4258,  0.3872,  1.4443,  ..., -3.8848,  1.9688, -1.6094],
        ...,
        [ 0.4258,  0.3872,  1.4443,  ..., -3.8848,  1.9688, -1.6094],
        [ 0.4258,  0.3872,  1.4443,  ..., -3.8848,  1.9688, -1.6094],
        [ 0.4258,  0.3872,  1.4443,  ..., -3.8848,  1.9688, -1.6094

In [3]:
# Below is the output of running the original llama code (llama_original.py) for reference
original_llm = LLM(model="meta-llama/Llama-2-7b-chat-hf", max_model_len=1024, enforce_eager=True)

INFO 03-23 00:27:17 config.py:520] This model supports multiple tasks: {'generate', 'reward', 'classify', 'embed', 'score'}. Defaulting to 'generate'.
WARNING 03-23 00:27:17 cuda.py:100] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 03-23 00:27:17 config.py:656] Async output processing is not supported on the current platform type cuda.
INFO 03-23 00:27:17 llm_engine.py:232] Initializing an LLM engine (v0.7.0) with config: model='meta-llama/Llama-2-7b-chat-hf', speculative_config=None, tokenizer='meta-llama/Llama-2-7b-chat-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 03-23 00:27:26 model_runner.py:1115] Loading model weights took 12.5523 GB
[LlamaForCausalLM forward] Input_ids shape: torch.Size([2048])
[LlamaModel forward] Input_ids shape: torch.Size([2048])
[LlamaForCausalLM forward] Model output shape: torch.Size([2048, 4096])
[compute_logits] Hidden states shape: torch.Size([2048, 4096])
[compute_logits] logits: tensor([[-3.4492,  5.1875,  3.8145,  ...,  0.0423, -1.7881,  0.1304],
        [-3.4492,  5.1875,  3.8145,  ...,  0.0423, -1.7881,  0.1304],
        [-3.4492,  5.1875,  3.8145,  ...,  0.0423, -1.7881,  0.1304],
        ...,
        [-3.4492,  5.1875,  3.8145,  ...,  0.0423, -1.7881,  0.1304],
        [-3.4492,  5.1875,  3.8145,  ...,  0.0423, -1.7881,  0.1304],
        [-3.4492,  5.1875,  3.8145,  ...,  0.0423, -1.7881,  0.1304]],
       device='cuda:0', dtype=torch.float16)
INFO 03-23 00:27:28 worker.py:266] Memory profiling takes 0.97 seconds
INFO 03-23 00:27:28 worker.py:266] the current vLLM instance can use total_gpu_memory (15.

In [ ]:
# Output of original lammama for reference
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 4/4 [00:00<00:00,  8.39it/s, est. speed input: 56.62 toks/s, output: 134.20 toks/s]

Prompt: 'Hello, my name is', Generated text: " Sherry and I'm a 35-year-old woman from"
Prompt: 'The president of the United States is', Generated text: ' a member of Congress, which means that he or she is subject to the same'
Prompt: 'The capital of France is', Generated text: ' Paris. This is a fact that is well known and widely accepted. However,'
Prompt: 'The future of AI is', Generated text: ' likely to be shaped by a combination of technological advancements, soci'


In [ ]:
# output of llama-ee (current llama.py is llama-ee)
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

input_ids shape before converting: torch.Size([27])
input_ids shape after converting: torch.Size([1, 27])
LlamaModel forward. input_ids shape: torch.Size([1, 27])
attention_mask shape: None
-------------------------
Is first rank. input ids shape: torch.Size([1, 27])
Force kv_caches to None
--------EE statistics---------
self.exited_rates: [0, 2]
skip_mask: False, conf: 0.0088043212890625
--------EE statistics---------
input_ids shape before converting: torch.Size([4])
input_ids shape after converting: torch.Size([1, 4])
LlamaModel forward. input_ids shape: torch.Size([1, 4])
attention_mask shape: None
-------------------------
Is first rank. input ids shape: torch.Size([1, 4])
Force kv_caches to None
--------EE statistics---------
self.exited_rates: [0, 3]
skip_mask: False, conf: 0.1258544921875
--------EE statistics---------
input_ids shape before converting: torch.Size([4])
input_ids shape after converting: torch.Size([1, 4])
LlamaModel forward. input_ids shape: torch.Size([1, 4])
a

Processed prompts: 100%|██████████| 4/4 [00:00<00:00,  4.47it/s, est. speed input: 30.20 toks/s, output: 71.59 toks/s]

--------EE statistics---------
self.exited_rates: [2, 11]
skip_mask: False, conf: 0.210693359375
--------EE statistics---------
input_ids shape before converting: torch.Size([4])
input_ids shape after converting: torch.Size([1, 4])
LlamaModel forward. input_ids shape: torch.Size([1, 4])
attention_mask shape: None
-------------------------
Is first rank. input ids shape: torch.Size([1, 4])
Force kv_caches to None
--------EE statistics---------
self.exited_rates: [2, 12]
skip_mask: False, conf: 0.394287109375
--------EE statistics---------
input_ids shape before converting: torch.Size([4])
input_ids shape after converting: torch.Size([1, 4])
LlamaModel forward. input_ids shape: torch.Size([1, 4])
attention_mask shape: None
-------------------------
Is first rank. input ids shape: torch.Size([1, 4])
Force kv_caches to None
--------EE statistics---------
self.exited_rates: [2, 13]
skip_mask: False, conf: 0.008636474609375
--------EE statistics---------
input_ids shape before converting: to

In [4]:
run_batch_inference(original_llm, "lmsys_1.jsonl", "results/lmsys_1_original.csv")

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[LlamaForCausalLM forward] Input_ids shape: torch.Size([13])
[LlamaModel forward] Input_ids shape: torch.Size([13])
[LlamaForCausalLM forward] Model output shape: torch.Size([13, 4096])
[compute_logits] Hidden states shape: torch.Size([13, 4096])
[compute_logits] logits: tensor([[-0.4729,  0.4666, 13.6406,  ...,  0.0463, -0.6255, -1.3252]],
       device='cuda:0', dtype=torch.float16)
[LlamaForCausalLM forward] Input_ids shape: torch.Size([1])
[LlamaModel forward] Input_ids shape: torch.Size([1])
[LlamaForCausalLM forward] Model output shape: torch.Size([1, 4096])
[compute_logits] Hidden states shape: torch.Size([1, 4096])
[compute_logits] logits: tensor([[-0.2966, -1.0352, 15.0625,  ...,  1.0303,  2.3867,  0.1385]],
       device='cuda:0', dtype=torch.float16)
[LlamaForCausalLM forward] Input_ids shape: torch.Size([1])
[LlamaModel forward] Input_ids shape: torch.Size([1])
[LlamaForCausalLM forward] Model output shape: torch.Size([1, 4096])
[compute_logits] Hidden states shape: torch.S

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.55s/it, est. speed input: 5.10 toks/s, output: 39.22 toks/s]

[LlamaForCausalLM forward] Model output shape: torch.Size([1, 4096])
[compute_logits] Hidden states shape: torch.Size([1, 4096])
[compute_logits] logits: tensor([[-4.2891, -4.1367,  4.3320,  ..., -1.7441, -4.8203, -3.4629]],
       device='cuda:0', dtype=torch.float16)
[LlamaForCausalLM forward] Input_ids shape: torch.Size([1])
[LlamaModel forward] Input_ids shape: torch.Size([1])
[LlamaForCausalLM forward] Model output shape: torch.Size([1, 4096])
[compute_logits] Hidden states shape: torch.Size([1, 4096])
[compute_logits] logits: tensor([[-5.7617, -8.2656,  5.6133,  ..., -2.9238, -1.3457, -3.2051]],
       device='cuda:0', dtype=torch.float16)
[LlamaForCausalLM forward] Input_ids shape: torch.Size([1])
[LlamaModel forward] Input_ids shape: torch.Size([1])
[LlamaForCausalLM forward] Model output shape: torch.Size([1, 4096])
[compute_logits] Hidden states shape: torch.Size([1, 4096])
[compute_logits] logits: tensor([[-3.6895, -9.6406,  4.3672,  ..., -0.9741, -2.7520, -3.0293]],
       

In [5]:
run_batch_inference(llm, "lmsys_1.jsonl", "results/lmsys_1.csv")

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

input_ids shape before converting: torch.Size([13])
input_ids shape after converting: torch.Size([13])
LlamaModel forward. input_ids shape: torch.Size([13])
attention_mask shape: None
-------------------------
Is first rank. input ids shape: torch.Size([13])
[compute_logits] hidden_states shape: torch.Size([13, 4096]). lm_head shape: ParallelLMHead(num_embeddings=32000, embedding_dim=4096, org_vocab_size=32000, num_embeddings_padded=32000, tp_size=1)
[compute_logits] logits: tensor([[ 4.9219,  2.2461, -4.3906,  ...,  3.3125,  2.5020,  2.0527],
        [ 0.1512,  1.1182,  1.6289,  ...,  2.8145,  0.8994, -3.0039],
        [-0.6582,  0.8691,  1.9893,  ...,  1.6553, -0.4167, -2.3047],
        ...,
        [-0.9692, -0.4407, -1.8320,  ...,  0.3606,  1.0049, -0.4836],
        [ 1.3262,  6.9922,  0.4502,  ..., -0.4648,  1.8838,  1.0205],
        [-1.6396, -1.3789,  3.7227,  ...,  0.9658, -1.4277, -3.4766]],
       device='cuda:0', dtype=torch.float16)
[get_skip_mask] mask dim: 1
--------[Llam

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.13s/it, est. speed input: 4.16 toks/s, output: 32.00 toks/s]

[compute_logits] hidden_states shape: torch.Size([1, 4096]). lm_head shape: ParallelLMHead(num_embeddings=32000, embedding_dim=4096, org_vocab_size=32000, num_embeddings_padded=32000, tp_size=1)
[compute_logits] logits: tensor([[-0.4443, -0.4487,  1.0566,  ..., -3.2539,  4.3516, -0.4888]],
       device='cuda:0', dtype=torch.float16)
--------[LlamaModel: forward] EE statistics at layer20---------
self.exited_rates([num_ee, num_no_ee]): [3, 92]
skip_mask: False, conf: 0.0054473876953125
--------[LlamaModel: forward] EE statistics---------
[LlamaModel: forward] Returning hidden_states shape: torch.Size([1, 4096])
[LlamaForCausalLM forward] Model output shape: torch.Size([1, 4096])
[compute_logits] hidden_states shape: torch.Size([1, 4096]). lm_head shape: ParallelLMHead(num_embeddings=32000, embedding_dim=4096, org_vocab_size=32000, num_embeddings_padded=32000, tp_size=1)
[compute_logits] logits: tensor([[-0.8618, -0.2849,  6.8242,  ..., -1.3984,  0.7227,  0.7676]],
       device='cuda:0